## Divide and Conquer Algorithms


### Organizing Lottery
You are organizing an online lottery. To participate, a person bets on a single integer. You then draw several segments of consecutive integers at random. A participant’s payoff is proportional to the number of segments that contain the participant’s number. You need an efficient algorithm for computing the payoffs for all participants. A simple scan of the list of all ranges for each participant is too slow since your lottery is very popular: you have thousands of participants and thousands of ranges.

Input: A list of **n≤50000** segments and a list of **m≤50000** points.

Output: The number of segments containing each point.

In [10]:
def points_cover_naive(starts, ends, points):
    assert len(starts) == len(ends)
    count = [0] * len(points)
    for index, point in enumerate(points):
        for start, end in zip(starts, ends):
            if start > end:
                start, end = end, start
            if start <= point <= end:
                count[index] += 1
    return count

In [2]:
def merge_sort(A):
    if len(A) == 1:
        return A
    m = len(A) // 2
    B = merge_sort(A[:m])
    C = merge_sort(A[m:])
    A_new = merge(B, C)
    return A_new

def merge(B, C):
    D = []
    while len(B) != 0 and len(C) != 0:
        if B[0] <= C[0]:
            D.append(B[0])
            B = B[1:]
        else:
            D.append(C[0])
            C = C[1:]
    D = D + B + C
    return D

def reverse_merge_sort(A):
    if len(A) == 1:
        return A
    m = len(A) // 2
    B = reverse_merge_sort(A[:m])
    C = reverse_merge_sort(A[m:])
    A_new = reverse_merge(B, C)
    return A_new

def reverse_merge(B, C):
    D = []
    while len(B) != 0 and len(C) != 0:
        if B[0][1] >= C[0][1]:
            D.append(B[0])
            B = B[1:]
        else:
            D.append(C[0])
            C = C[1:]
    D = D + B + C
    return D

def starting(A, x):
    if len(A) <= 1:
        return A
    mid = len(A) // 2
    left = A[:mid]
    right = A[mid:]
    if right[0][0] <= x:
        return left + starting(right, x)
    else:
        return starting(left, x)

def ending(A, x):
    if len(A) <= 1:
        return A
    mid = len(A) // 2
    left = A[:mid]
    right = A[mid:]
    if right[0][1] >= x:
        return left + ending(right, x)
    else:
        return ending(left, x)

def points_cover(starts, ends, points):
    assert len(starts) == len(ends)
    pair = []
    for i in range(len(starts)):
        if starts[i] <= ends[i]:
            pair.append([starts[i], ends[i]])
        else:
            pair.append([ends[i], starts[i]])
    # Start and End pairs are appended in the list
    sorted_pair = merge_sort(pair)
    # Pairs are sorted by increasing order
    print(f'Segments: {sorted_pair}')
    output_count = []
    for j in points:
        print(f"For point {j}:")
        start_extracted = starting(sorted_pair, j)
        # Some segments are extracted where left parts of the segments are bigger than the point
        start_extracted_reversed = reverse_merge_sort(start_extracted)
        # What is left from the segments are sorted by decreasing order of the right parts of the segments
        end_extracted = ending(start_extracted_reversed, j)
        # Some segments are extracted where right parts of the segments are smaller than the point
        print(f'Counting segments: {end_extracted}')
        output_count.append(len(end_extracted))
        print(f'Total number of segments including {j} is: {output_count[-1]}')
    return output_count

if __name__ == '__main__':
    data = list(map(int, input().split()))
    n, m = data[0], data[1]
    input_starts, input_ends = data[2:2 * n + 2:2], data[3:2 * n + 2:2]
    input_points = data[2 * n + 2:]

    output_count = points_cover(input_starts, input_ends, input_points)
    print(*output_count)

9 9 10 12 1 3 8 10 7 11 10 12 4 5 2 15 9 16 2 7 4 1 6 7 13 8 3 10
Segments: [[1, 3], [2, 7], [2, 15], [4, 5], [7, 11], [8, 10], [9, 16], [10, 12], [10, 12]]
For point 4:
Counting segments: [[2, 15], [2, 7], [4, 5]]
Total number of segments including 4 is: 3
For point 1:
Counting segments: [[1, 3]]
Total number of segments including 1 is: 1
For point 6:
Counting segments: [[2, 15], [2, 7]]
Total number of segments including 6 is: 2
For point 7:
Counting segments: [[2, 15], [7, 11], [2, 7]]
Total number of segments including 7 is: 3
For point 13:
Counting segments: [[9, 16], [2, 15]]
Total number of segments including 13 is: 2
For point 8:
Counting segments: [[2, 15], [7, 11], [8, 10]]
Total number of segments including 8 is: 3
For point 3:
Counting segments: [[2, 15], [2, 7], [1, 3]]
Total number of segments including 3 is: 3
For point 10:
Counting segments: [[9, 16], [2, 15], [10, 12], [10, 12], [7, 11], [8, 10]]
Total number of segments including 10 is: 6
3 1 2 3 2 3 3 6


    With large input, the algorithm was slow. So, while the complexity was m*n in the naive version, it was m*n*logn in the above version. So I tried to improve it.

In [3]:
def points_cover(starts, ends, points):
    assert len(starts) == len(ends)
    pair = []
    for i in range(len(starts)):
        if starts[i] <= ends[i]:
            pair.append([starts[i], ends[i]])
        else:
            pair.append([ends[i], starts[i]])
    segments = merge_sort(pair)
    reverse_segments = reverse_merge_sort(pair)
    output_count = []
    for j in points:
        result = []
        count = 0
        start_extracted = starting(segments, j)
        end_extracted = ending(reverse_segments, j)
        if len(start_extracted) <= len(end_extracted):
            for k in start_extracted:
                if k[0] < j < k[1]:
                    count += 1
            result.append(count)
        else:            
            for k in end_extracted:
                if k[0] < j < k[1]:
                    count += 1
            result.append(count)
    return result

    Although some undesired parts are removed with merge sort and recursive binary search algorithm, complexity is again m*n and the code does not work better than the naive algorithm. So I tried again to find the best possible solution.

In [28]:
def points_cover(starts, ends, points):
    assert len(starts) == len(ends)
    pair = []
    for i in range(len(starts)):
        if starts[i] <= ends[i]:
            pair.append((starts[i], 'left'))
            pair.append((ends[i], 'right'))
        else:
            pair.append((ends[i], 'left'))     
            pair.append((starts[i], 'right'))
    for j in range(len(points)):
        pair.append((points[j], 'point'))
    pair.sort()
    result_dictionary = {}
    count_left = 0
    count_right = 0
    for k in pair:
        if k[1] == 'left':
            count_left += 1
        elif k[1] == 'right':
            count_right += 1
        else:
            result_dictionary[f'point {k[0]}'] = count_left - count_right
    # print(result_dictionary)
    final_result = []
    for z in points:
        final_result.append(result_dictionary[f'point {z}'])
    print(f'points: {points}')
    return final_result

if __name__ == '__main__':
    data = list(map(int, input().split()))
    n, m = data[0], data[1]
    input_starts, input_ends = data[2:2 * n + 2:2], data[3:2 * n + 2:2]
    input_points = data[2 * n + 2:]

    output_count = points_cover(input_starts, input_ends, input_points)
    print(f'counts: {output_count}')

In [26]:
import random
import time

def StressTest(N, M, L):
    assert 1 <= N <= 5 * 10**4
    assert 1 <= M <= 5 * 10**4
    assert 0 <= L <= 10**8
    while True:
        n = random.randint(1, N)
        m = random.randint(1, M)
        A = [random.randint(0, L) for i in range(0, n)]
        B = [random.randint(0, L) for i in range(0, n)]
        C = [random.randint(0, L) for i in range(0, m)]
        print(A)
        print(B)
        print(C)
        result1 = points_cover_naive(A, B, C)
        result2 = points_cover(A, B, C)
        if result1 == result2:
            print('OK', 'result1, result2 = ', result1)
        else:
            print('Answer is wrong:', 'result1 = ', result1, 'result2 = ', result2)
            break
            
def ComplexityTest(N, M, L):
    assert 1 <= N <= 5 * 10**4
    assert 1 <= M <= 5 * 10**4
    assert 0 <= L <= 10**8
    A = [random.randint(0, L) for i in range(0, N)]
    B = [random.randint(0, L) for i in range(0, N)]
    C = [random.randint(0, L) for i in range(0, M)]
    start_time = time.time()
    points_cover(A, B, C)
    print("--- %s seconds ---" % (time.time() - start_time))

In [13]:
StressTest(20, 20, 1000)

[628, 60, 888, 645, 681, 447, 15, 785, 785, 577, 223, 55, 415, 11, 996, 702, 650]
[964, 511, 969, 134, 539, 773, 987, 463, 632, 622, 113, 772, 660, 96, 779, 377, 536]
[702, 843, 171, 470, 412, 376, 440, 849, 228, 701, 453, 521, 401]
points: [702, 843, 171, 470, 412, 376, 440, 849, 228, 701, 453, 521, 401]
OK result1, result2 =  [7, 3, 5, 8, 5, 4, 6, 3, 4, 7, 7, 7, 5]
[878, 854, 426, 647, 679, 3]
[447, 904, 95, 579, 222, 230]
[231, 176, 381, 567, 408, 719, 135, 392, 690, 319, 675, 824, 636, 950, 951, 828, 27]
points: [231, 176, 381, 567, 408, 719, 135, 392, 690, 319, 675, 824, 636, 950, 951, 828, 27]
OK result1, result2 =  [2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 3, 0, 0, 1, 1]
[227, 400, 907, 165, 165, 873, 349, 317, 491, 592, 106, 841]
[51, 962, 682, 57, 81, 123, 909, 801, 641, 676, 58, 910]
[387, 464, 418, 28, 135, 24, 808, 169, 321, 938, 915, 535]
points: [387, 464, 418, 28, 135, 24, 808, 169, 321, 938, 915, 535]
OK result1, result2 =  [3, 4, 4, 0, 4, 0, 4, 2, 2, 1, 1, 5]
[83, 70, 320, 

[332, 215, 453, 801, 179, 188, 520, 408, 40, 697, 392, 263]
[210, 438, 889, 156, 978, 583, 198, 656, 368, 478, 417, 275]
[397, 748, 717, 357, 952, 685, 703, 312, 725, 742, 991, 342, 346, 332]
points: [397, 748, 717, 357, 952, 685, 703, 312, 725, 742, 991, 342, 346, 332]
OK result1, result2 =  [6, 3, 3, 6, 1, 4, 3, 7, 3, 3, 0, 6, 6, 7]
[524, 969, 529, 12, 891, 303, 537, 598, 840, 622, 912, 987]
[102, 60, 1, 481, 426, 877, 729, 58, 453, 841, 665, 221]
[20, 741, 909, 683, 922, 688, 356, 405, 316, 554, 792, 539, 889, 31]
points: [20, 741, 909, 683, 922, 688, 356, 405, 316, 554, 792, 539, 889, 31]
OK result1, result2 =  [2, 7, 3, 8, 2, 8, 7, 7, 7, 7, 7, 7, 4, 2]
[62, 519, 257, 829, 533, 973, 636, 877]
[490, 555, 68, 866, 458, 241, 684, 427]
[543, 449, 387, 299, 644, 951, 712, 563, 948, 989, 152, 956]
points: [543, 449, 387, 299, 644, 951, 712, 563, 948, 989, 152, 956]
OK result1, result2 =  [3, 3, 2, 2, 3, 1, 2, 2, 1, 0, 2, 1]
[764, 448, 941]
[521, 453, 599]
[980, 426, 995, 518, 211, 642, 2

KeyboardInterrupt: 

In [25]:
ComplexityTest(50000, 1000, 1000) # naive algorithm

--- 9.490639925003052 seconds ---


In [33]:
ComplexityTest(50000, 50000, 1000000) # fast algorithm

--- 0.274843692779541 seconds ---
